## Analyze UniProt data
- see what type of data is available in the uniprot bulk download
  - how much
  - what are the sources of evidence

In [1]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm
import re

In [104]:
uniprot_dat_file = "/projects/deepgreen/jlaw/inputs/uniprot/2022-03/uniprot_sprot.dat.gz"
# uniprot_dat_file = "/projects/deepgreen/jlaw/inputs/uniprot/2022-03/P09884.txt"

In [2]:
parsed_uniprot_dat_file = "/Users/jlaw/Downloads/lab/uniprot/uniprot_seq_features.csv.gz"
data = pd.read_csv(parsed_uniprot_dat_file)
data.head(2)

,taxon_id,uniprot_id,feature,sites,id,note,evidence,sources
0,654924,Q6GZX4,CHAIN,1..256,PRO_0000410512,Putative transcription factor 001R,NaN,NaN
1,654924,Q6GZX3,CHAIN,1..320,PRO_0000410509,Uncharacterized protein 002L,NaN,NaN


In [4]:
data['feature'].value_counts()

CHAIN       575185
BINDING     419187
REGION      414742
METAL       411532
TRANSMEM    377972
STRAND      301247
HELIX       293064
MOD_RES     245795
DOMAIN      210221
COMPBIAS    172455
ACT_SITE    170197
NP_BIND     164421
TOPO_DOM    147049
CONFLICT    137532
DISULFID    131053
CARBOHYD    121836
REPEAT      108235
VARIANT      93863
MUTAGEN      78647
TURN         70771
SITE         62907
VAR_SEQ      51750
MOTIF        46253
SIGNAL       43515
ZN_FING      30350
CROSSLNK     23223
COILED       22297
INIT_MET     17469
PROPEP       14919
NON_TER      12622
PEPTIDE      12199
DNA_BIND     12063
LIPID        11237
TRANSIT       9401
UNSURE        5695
INTRAMEM      2997
NON_CONS      2539
CA_BIND       2355
NON_STD        358
Name: feature, dtype: int64

In [6]:
data2 = data.dropna(subset=["evidence"])

In [7]:
data2['feature'].value_counts()

BINDING     415655
METAL       402591
REGION      389250
TRANSMEM    376710
STRAND      301247
HELIX       293064
MOD_RES     245520
DOMAIN      184706
COMPBIAS    172455
ACT_SITE    168406
NP_BIND     163380
TOPO_DOM    145901
CONFLICT    136515
DISULFID    127234
CARBOHYD    120955
MUTAGEN      76892
TURN         70771
VARIANT      60768
SITE         59849
VAR_SEQ      49466
SIGNAL       41808
ZN_FING      28791
MOTIF        26255
CROSSLNK     22744
COILED       22191
REPEAT       19490
CHAIN        18836
INIT_MET     16774
PROPEP       12682
DNA_BIND     11487
LIPID        11237
TRANSIT       8978
PEPTIDE       5651
UNSURE        4652
INTRAMEM      2900
NON_CONS      2533
NON_TER       2091
CA_BIND       1535
NON_STD        177
Name: feature, dtype: int64

In [14]:
data2[data2['feature'] == 'BINDING'].note.value_counts().head(20)

Substrate                                       106099
ATP                                              46918
S-adenosyl-L-methionine                          21039
NAD                                              17917
Pyridoxal phosphate                              11100
Substrate; via amide nitrogen                     7442
NADP                                              7432
S-adenosyl-L-methionine; via carbonyl oxygen      7130
FMN                                               6183
UDP-GlcNAc                                        5603
ATP; via amide nitrogen                           5243
FAD                                               4688
AMP                                               4678
GTP                                               4586
S-adenosyl-L-methionine; via amide nitrogen       4554
Citrulline                                        4373
Aspartate                                         4159
ATP; via amide nitrogen and carbonyl oxygen       3669
IMP       

In [156]:
# now try limiting to manual evidence codes 
manual_codes = set([
    "ECO:0000269",  # Experimental
    "ECO:0000303",  # Non-traceable author statement
    "ECO:0000305",  # Curator inference
    "ECO:0000250",  # Sequence similarity
    "ECO:0000255",  # Sequence model. Source: A rule (optional)
    "ECO:0007744",  # Combinatorial
    "ECO:0000312",  # Imported information
])
# experimental codes
exp_codes = set([
    "ECO:0000269",  # Experimental
    "ECO:0000303",  # Non-traceable author statement
    "ECO:0000305",  # Curator inference
    "ECO:0007744",  # Combinatorial
    "ECO:0000312",  # Imported information
])
# manual codes - Rules
exp_seq_codes = set([
    "ECO:0000269",  # Experimental
    "ECO:0000303",  # Non-traceable author statement
    "ECO:0000305",  # Curator inference
    "ECO:0007744",  # Combinatorial
    "ECO:0000312",  # Imported information
    "ECO:0000250",  # Sequence similarity
])
# these two have only structure annotations e.g., helix, turn, strand
#     "ECO:0007829",  # PDB entries 
#     "ECO:0000313",  # An entry from nucleotide sequence databases (EMBL, Ensembl), PDB, model organism databases

In [157]:
# check if the set of evidence codes overlap
data2['manual_code'] = data2['evidence'].apply(lambda x: len(set(x.split(',')) & manual_codes) > 0)
data2['exp_code'] = data2['evidence'].apply(lambda x: len(set(x.split(',')) & exp_codes) > 0)
data2['es_code'] = data2['evidence'].apply(lambda x: len(set(x.split(',')) & exp_seq_codes) > 0)
data2.head()

<ipython-input-157-5a1498d9b077>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['manual_code'] = data2['evidence'].apply(lambda x: len(set(x.split(',')) & manual_codes) > 0)
<ipython-input-157-5a1498d9b077>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['exp_code'] = data2['evidence'].apply(lambda x: len(set(x.split(',')) & exp_codes) > 0)
<ipython-input-157-5a1498d9b077>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,taxon_id,uniprot_id,feature,sites,id,note,evidence,sources,manual_code,exp_code,ei_code,es_code
2,654924,Q6GZX3,TRANSMEM,301..318,NaN,Helical,ECO:0000255,NaN,True,False,False,False
3,654924,Q6GZX3,REGION,261..294,NaN,Disordered,ECO:0000256,SAM:MobiDB-lite,False,False,False,False
5,345201,Q197F8,REGION,339..397,NaN,Disordered,ECO:0000256,SAM:MobiDB-lite,False,False,False,False
6,345201,Q197F8,REGION,434..458,NaN,Disordered,ECO:0000256,SAM:MobiDB-lite,False,False,False,False
7,345201,Q197F8,COMPBIAS,348..387,NaN,Acidic residues,ECO:0000256,SAM:MobiDB-lite,False,False,False,False


In [160]:
man_data = data2[data2.manual_code]
print(f"{len(man_data)} / {len(data2)} have a manual evidence code")
exp_data = data2[data2.exp_code]
print(f"{len(exp_data)} / {len(data2)} have an experimental evidence code")
es_data = data2[data2.es_code]
print(f"{len(es_data)} / {len(data2)} have an experimental evidence code or manual sequence similarity")

3181868 / 4222147 have a manual evidence code
559111 / 4222147 have an experimental evidence code
1185669 / 4222147 have an experimental evidence code or manual sequence similarity


In [131]:
man_data.feature.value_counts().sort_index()

ACT_SITE    168406
BINDING     415655
CARBOHYD    120955
CA_BIND       1535
CHAIN        18836
COILED       22191
CONFLICT    136515
CROSSLNK     22744
DISULFID    127234
DNA_BIND     11487
DOMAIN      184706
HELIX            4
INIT_MET     16774
INTRAMEM      2900
LIPID        11237
METAL       402591
MOD_RES     245520
MOTIF        26255
MUTAGEN      76892
NON_CONS      2533
NON_STD        177
NON_TER       2091
NP_BIND     163380
PEPTIDE       5651
PROPEP       12682
REGION      186503
REPEAT       19490
SIGNAL       41808
SITE         59849
STRAND           1
TOPO_DOM    145901
TRANSIT       8978
TRANSMEM    376710
UNSURE        4652
VARIANT      60768
VAR_SEQ      49466
ZN_FING      28791
Name: feature, dtype: int64

In [130]:
man_data.groupby('feature').uniprot_id.nunique()

feature
ACT_SITE    102109
BINDING     112935
CARBOHYD     30650
CA_BIND        817
CHAIN        11457
COILED       15357
CONFLICT     47738
CROSSLNK      8556
DISULFID     33915
DNA_BIND     10352
DOMAIN      116210
HELIX            2
INIT_MET     16729
INTRAMEM      1349
LIPID         8675
METAL        99304
MOD_RES      73692
MOTIF        17718
MUTAGEN      17963
NON_CONS       819
NON_STD        157
NON_TER       1631
NP_BIND      87884
PEPTIDE       3999
PROPEP       11318
REGION       89017
REPEAT        2444
SIGNAL       41807
SITE         32151
STRAND           1
TOPO_DOM     29680
TRANSIT       8875
TRANSMEM     78939
UNSURE         564
VARIANT      10949
VAR_SEQ      21307
ZN_FING      12025
Name: uniprot_id, dtype: int64

In [153]:
ei_data.feature.value_counts().sort_index()

ACT_SITE      4372
BINDING      12731
CARBOHYD      9093
CA_BIND        153
CHAIN         3407
COILED         130
CONFLICT    136503
CROSSLNK      7618
DISULFID     12907
DNA_BIND       139
DOMAIN        1083
HELIX       293064
INIT_MET      6179
INTRAMEM       175
LIPID         1480
METAL        12955
MOD_RES      90952
MOTIF         3433
MUTAGEN      76891
NON_CONS      2533
NON_STD         85
NON_TER       2088
NP_BIND       3940
PEPTIDE       3913
PROPEP        3791
REGION       11637
REPEAT        3707
SIGNAL        4281
SITE          3808
STRAND      301247
TOPO_DOM     19343
TRANSIT       1280
TRANSMEM      3458
TURN         70771
UNSURE        4651
VARIANT      60768
VAR_SEQ      49431
ZN_FING        191
Name: feature, dtype: int64

In [154]:
ei_data.groupby('feature').uniprot_id.nunique()

feature
ACT_SITE     2780
BINDING      3658
CARBOHYD     3333
CA_BIND        73
CHAIN        3010
COILED        101
CONFLICT    47735
CROSSLNK     2665
DISULFID     4310
DNA_BIND       95
DOMAIN        670
HELIX       26396
INIT_MET     6148
INTRAMEM       93
LIPID        1123
METAL        2869
MOD_RES     27126
MOTIF        2385
MUTAGEN     17963
NON_CONS      819
NON_STD        67
NON_TER      1629
NP_BIND      1628
PEPTIDE      3163
PROPEP       3228
REGION       5803
REPEAT        428
SIGNAL       4281
SITE         1795
STRAND      24848
TOPO_DOM     3401
TRANSIT      1274
TRANSMEM      782
TURN        21476
UNSURE        563
VARIANT     10949
VAR_SEQ     21272
ZN_FING       128
Name: uniprot_id, dtype: int64

In [102]:
cols = exp_data.feature.value_counts().index
i = -1
exp_data.feature.value_counts()

CONFLICT    136503
MOD_RES      90952
MUTAGEN      76891
VARIANT      60768
VAR_SEQ      49431
TOPO_DOM     19343
METAL        12955
DISULFID     12907
BINDING      12731
REGION       11637
CARBOHYD      9093
CROSSLNK      7618
INIT_MET      6179
UNSURE        4651
ACT_SITE      4372
SIGNAL        4281
NP_BIND       3940
PEPTIDE       3913
SITE          3808
PROPEP        3791
REPEAT        3707
TRANSMEM      3458
MOTIF         3433
CHAIN         3407
NON_CONS      2533
NON_TER       2088
LIPID         1480
TRANSIT       1280
DOMAIN        1083
ZN_FING        191
INTRAMEM       175
CA_BIND        153
DNA_BIND       139
COILED         130
NON_STD         85
HELIX            4
STRAND           1
Name: feature, dtype: int64

In [128]:
exp_data.groupby('feature').uniprot_id.nunique()

feature
ACT_SITE     2780
BINDING      3658
CARBOHYD     3333
CA_BIND        73
CHAIN        3010
COILED        101
CONFLICT    47735
CROSSLNK     2665
DISULFID     4310
DNA_BIND       95
DOMAIN        670
HELIX           2
INIT_MET     6148
INTRAMEM       93
LIPID        1123
METAL        2869
MOD_RES     27126
MOTIF        2385
MUTAGEN     17963
NON_CONS      819
NON_STD        67
NON_TER      1629
NP_BIND      1628
PEPTIDE      3163
PROPEP       3228
REGION       5803
REPEAT        428
SIGNAL       4281
SITE         1795
STRAND          1
TOPO_DOM     3401
TRANSIT      1274
TRANSMEM      782
UNSURE        563
VARIANT     10949
VAR_SEQ     21272
ZN_FING       128
Name: uniprot_id, dtype: int64

In [214]:
man_data[(man_data.feature == "BINDING") & (man_data.note == "NADP")].sources.astype(str).apply(lambda x: x.split(':')[0]).value_counts(dropna=False).head(20)

HAMAP-Rule    5014
UniProtKB     1251
nan            780
PubMed         311
PDB             12
Ref.9           11
Ref.17           8
Ref.4            8
Ref.11           5
Ref.7,PDB        4
Ref.2,PDB        4
Ref.7            4
Ref.2            3
Ref.18           3
Ref.14           3
Ref.3            3
Ref.12           3
Ref.4,PDB        2
Ref.8,PDB        2
Ref.18,PDB       1
Name: sources, dtype: int64

In [216]:
man_data[(man_data.feature == "TRANSMEM")].sources.astype(str).apply(lambda x: x.split(':')[0]).value_counts(dropna=False).head(20)

nan                  206434
HAMAP-Rule           135272
UniProtKB             24329
PROSITE-ProRule        7756
PubMed                 2107
,PROSITE- ProRule       679
,PubMed                  62
PDB                      44
Ref.55                   12
,                        12
Ref.11                    1
Ref.1                     1
RuleBase                  1
Name: sources, dtype: int64

In [149]:
man_data[(man_data.feature == "BINDING") & (man_data.note == "NAD") & (man_data.sources.str.contains("Rule")==False)].sources.astype(str).apply(lambda x: x.split(':')[0]).value_counts(dropna=False).head(20)

UniProtKB     1719
PubMed         362
PIRSR           17
Ref.3           16
PDB              7
Ref.12           6
Ref.11           6
Ref.2            5
Ref.13,PDB       4
Ref.2,PDB        4
Ref.21           3
Ref.6            3
Ref.16           3
Ref.6,PDB        3
Ref.18,PDB       2
Ref.20           2
Ref.4            2
Ref.3,PDB        2
Ref.5,PDB        1
Ref.9            1
Name: sources, dtype: int64

In [36]:
exp_data2 = exp_data.drop(axis=1, columns=['taxon_id', 'manual_code', 'exp_code'])
exp_data2.head(2)

,uniprot_id,feature,sites,id,note,evidence,sources
277,Q6GZQ0,REGION,39..61,NaN,Membrane-binding amphipathic helix,ECO:0000305,NaN
570,A9JLI2,SIGNAL,1..26,NaN,NaN,ECO:0000269,PubMed:30279544


In [166]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Missing (in Ref. 2; AA sequence)    124
N -> D (in Ref. 2; AA sequence)      97
Missing (in Ref. 1)                  85
D -> N (in Ref. 2; AA sequence)      77
Missing (in Ref. 3; AA sequence)     71
Q -> E (in Ref. 2; AA sequence)      64
E -> Q (in Ref. 2; AA sequence)      62
Missing (in Ref. 1; AA sequence)     60
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
846,P13744,CONFLICT,27,NaN,S -> E (in Ref. 2; AA sequence),ECO:0000305,NaN
847,P13744,CONFLICT,30,NaN,E -> S (in Ref. 2; AA sequence),ECO:0000305,NaN
922,P32234,CONFLICT,2,NaN,S -> I (in Ref. 1; CAA50701),ECO:0000305,NaN


In [167]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Phosphoserine             52058
Phosphothreonine           8622
N6-acetyllysine            3588
Phosphotyrosine            1881
N-acetylalanine            1570
4-hydroxyproline           1340
N-acetylmethionine         1302
Omega-N-methylarginine     1002
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
843,P13744,MOD_RES,22,NaN,Pyrrolidone carboxylic acid,ECO:0000269,Ref.2
1093,P42643,MOD_RES,2,NaN,N-acetylalanine,ECO:0007744,PubMed:22223895
1097,P42643,MOD_RES,267,NaN,Phosphoserine,ECO:0007744,PubMed:19376835


In [168]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

D->A: Loss of activity.    128
H->A: Loss of activity.    116
S->A: No effect.            88
C->S: Loss of activity.     82
R->A: Loss of activity.     70
C->A: Loss of activity.     68
E->A: Loss of activity.     66
S->A: Loss of activity.     59
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1128,Q20655,MUTAGEN,240,NaN,D->E: Loss of ced-3-mediated cleavage.,ECO:0000269,PubMed:17371877
1210,P48349,MUTAGEN,61,NaN,S->A: Normal phosphorylation.,ECO:0000269,PubMed:28344081
1211,P48349,MUTAGEN,70,NaN,S->A: Reduced phosphorylation by CRPK1. Impair...,ECO:0000269,PubMed:28344081


In [169]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Missing             122
A -> V               42
V -> A               41
I -> V               38
Missing (in MFS)     37
T -> A               35
A -> T               33
L -> P               32
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1301,P31946,VARIANT,99,VAR_064762,V -> I (found in a renal cell carcinoma sample...,ECO:0000269,PubMed:21248752
1672,P61981,VARIANT,15,VAR_080224,E -> A (in DEE56; unknown pathological signifi...,ECO:0000269,PubMed:28777935
1673,P61981,VARIANT,50,VAR_080225,K -> Q (found in an individual with autism; un...,ECO:0000269,PubMed:28777935


In [170]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Missing (in isoform 2)                  12213
Missing (in isoform 3)                   4659
Missing (in isoform 4)                   1978
Missing (in isoform 5)                   1003
Missing (in isoform 2 and isoform 3)      603
Missing (in isoform b)                    506
Missing (in isoform 6)                    482
Missing (in isoform 3 and isoform 4)      297
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1075,P48347,VAR_SEQ,254,VSP_008972,N -> V (in isoform 2),ECO:0000303,PubMed:14593172
1080,Q9S9Z8,VAR_SEQ,241,VSP_042043,E -> K (in isoform 2),ECO:0000303,Ref.1
1081,Q9S9Z8,VAR_SEQ,242..252,VSP_042044,Missing (in isoform 2),ECO:0000303,Ref.1


In [171]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Cytoplasmic                    9526
Extracellular                  5048
Lumenal                        2439
Periplasmic                    1244
Mitochondrial intermembrane     405
Mitochondrial matrix            323
Vacuolar                         77
Stromal                          54
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
13183,P08195,TOPO_DOM,102..184,NaN,Cytoplasmic,ECO:0000269,PubMed:30867591
13185,P08195,TOPO_DOM,206..630,NaN,Extracellular,ECO:0000269,PubMed:30867591
16995,P41595,TOPO_DOM,1..56,NaN,Extracellular,"ECO:0000269,ECO:0000269,ECO:0000269","PubMed:23519215,PubMed:24357322,PubMed:28129538"


In [172]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Zinc                    1177
Magnesium                770
Zinc 2                   542
Calcium                  522
Zinc 1                   449
Zinc; catalytic          434
Calcium 2                391
Iron-sulfur (4Fe-4S)     312
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
5633,Q7NXD4,METAL,7,NaN,Manganese 1; via tele nitrogen,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"
5634,Q7NXD4,METAL,9,NaN,Manganese 1; via tele nitrogen,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"
5635,Q7NXD4,METAL,14,NaN,Manganese 2,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"


In [173]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN                                            11767
Interchain                                       321
Redox-active                                     120
Alternate                                         30
Interchain (with heavy chain)                     30
Interchain (between light and heavy chains)       22
Interchain (between B and A chains)               20
Interchain (with a heavy chain)                   14
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
844,P13744,DISULFID,48..81,NaN,NaN,ECO:0000269,Ref.3
845,P13744,DISULFID,124..303,NaN,Interchain (between gamma and delta chains),ECO:0000269,Ref.3
5177,P04403,DISULFID,40..92,NaN,Interchain (between small and large chains),ECO:0000269,PubMed:12421566


In [174]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Substrate                        2544
ATP                               851
NAD                               527
FAD                               497
NADP                              433
S-adenosyl-L-methionine           371
FMN                               175
Substrate; via amide nitrogen     157
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
2678,P37821,BINDING,145,NaN,Substrate,"ECO:0000305,ECO:0007744,ECO:0007744","PubMed:12686108,PDB:1B8G,PDB:1M4N"
2679,P37821,BINDING,151,NaN,Substrate,"ECO:0000305,ECO:0007744,ECO:0007744","PubMed:12686108,PDB:1B8G,PDB:1M4N"
5642,Q7NXD4,BINDING,14,NaN,Substrate,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"


In [175]:
feat_data = exp_data2[exp_data2.feature == "BINDING"]
print(feat_data['note'].value_counts(dropna=False).head(30))
# feat_data.head(3)

Substrate                                       2544
ATP                                              851
NAD                                              527
FAD                                              497
NADP                                             433
S-adenosyl-L-methionine                          371
FMN                                              175
Substrate; via amide nitrogen                    157
Pyridoxal phosphate                              153
Heme; covalent                                   127
AMP                                              123
FAD; via amide nitrogen                          121
S-adenosyl-L-methionine; via carbonyl oxygen     119
Glutathione                                      118
FAD; via amide nitrogen and carbonyl oxygen      113
GTP                                              109
Substrate; via carbonyl oxygen                   101
NAD; via carbonyl oxygen                          96
Coenzyme A                                    

In [176]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Substrate binding                            1002
S-adenosyl-L-methionine binding               245
Ketosynthase (KS) domain                      107
Glutathione binding                           106
Malonyl-CoA:ACP transacylase (MAT) domain     105
Acetyl-CoA binding                             86
IgE-binding                                    71
Pyridoxal phosphate binding                    69
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
277,Q6GZQ0,REGION,39..61,NaN,Membrane-binding amphipathic helix,ECO:0000305,NaN
792,B5KVH4,REGION,118..132,NaN,IgE-binding,ECO:0000269,PubMed:21718052
794,B5KVH4,REGION,208..219,NaN,IgE-binding,ECO:0000269,PubMed:21718052


In [177]:
feat_data = exp_data2[exp_data2.feature == "REGION"]
print(feat_data['note'].value_counts(dropna=False).head(30))

Substrate binding                                           1002
S-adenosyl-L-methionine binding                              245
Ketosynthase (KS) domain                                     107
Glutathione binding                                          106
Malonyl-CoA:ACP transacylase (MAT) domain                    105
Acetyl-CoA binding                                            86
IgE-binding                                                   71
Pyridoxal phosphate binding                                   69
Calmodulin-binding                                            66
Ketoreductase (KR) domain                                     55
Thioesterase (TE) domain                                      43
Adenylation 1                                                 43
Linker                                                        43
Condensation 2                                                42
Condensation 1                                                42
Adenylation 2            

In [178]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

N-linked (GlcNAc...) asparagine              6085
O-linked (GalNAc...) threonine                483
N-linked (GlcNAc...) (complex) asparagine     259
N-linked (GlcNAc...) asparagine; by host      214
O-linked (GalNAc...) serine                   204
O-linked (Ara...) hydroxyproline              129
C-linked (Man) tryptophan                     109
C-linked (Man) hydroxytryptophan               90
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
3583,P22283,CARBOHYD,71,NaN,N-linked (GlcNAc...) asparagine,ECO:0000305,NaN
10782,P01446,CARBOHYD,29,NaN,N-linked (GlcNAc...) asparagine,ECO:0000269,PubMed:15128311
13197,P08195,CARBOHYD,365,NaN,N-linked (GlcNAc...) asparagine,"ECO:0000269,ECO:0000269,ECO:0000269,ECO:000026...","PubMed:12754519,PubMed:19159218,PubMed:1934997..."


In [179]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in SUMO2)                   4686
Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in ubiquitin)               1300
Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in SUMO2); alternate         477
Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in SUMO)                     167
Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in ubiquitin); alternate     143
Cyclopeptide (Gly-Asn)                                                                  111
Isoglutamyl lysine isopeptide (Lys-Gln) (interchain with Q-Cter in protein Pup)         111
Glycyl lysine isopeptide (Lys-Gly) (interchain with G-Cter in SUMO1)                    105
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1504,P62258,CROSSLNK,50,NaN,Glycyl lysine isopeptide (Lys-Gly) (interchain...,ECO:0007744,PubMed:28112733
1841,P27348,CROSSLNK,49,NaN,Glycyl lysine isopeptide (Lys-Gly) (interchain...,ECO:0007744,PubMed:28112733
6486,P85078,CROSSLNK,1..9,NaN,Isoaspartyl cysteine isopeptide (Cys-Asn),ECO:0000269,PubMed:8689231


In [180]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Removed               6008
Removed; alternate      94
Removed; by host        73
Removed; partial         3
Removed; Partial         1
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
934,Q8GBW6,INIT_MET,1,NaN,Removed,ECO:0000269,PubMed:8366018
1091,P42643,INIT_MET,1,NaN,Removed,ECO:0007744,PubMed:22223895
1172,P46077,INIT_MET,1,NaN,Removed,ECO:0007744,PubMed:22223895


In [181]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

L or I                                   2429
I or L                                   1287
K or Q                                    559
Q or K                                    208
Assigned by comparison with orthologs      44
NaN                                        39
I OR L                                      7
Assigned by similarity to orthologs         5
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
9114,C0HJW9,UNSURE,3,NaN,NaN,ECO:0000305,PubMed:26293154
9115,C0HJW9,UNSURE,6,NaN,NaN,ECO:0000305,PubMed:26293154
9116,C0HJW9,UNSURE,11,NaN,NaN,ECO:0000305,PubMed:26293154


In [182]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN                                 1138
Proton acceptor                      611
Nucleophile                          596
Charge relay system                  454
Proton donor                         426
Proton donor/acceptor                222
Phosphocysteine intermediate          63
4-aspartylphosphate intermediate      41
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
4949,A5CAL1,ACT_SITE,232,NaN,NaN,ECO:0000305,PubMed:31488549
4950,A5CAL1,ACT_SITE,261,NaN,NaN,ECO:0000305,PubMed:31488549
4951,A5CAL1,ACT_SITE,279,NaN,Proton donor,ECO:0000305,PubMed:31488549


In [183]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN                          4207
Tat-type signal                62
Not cleaved                     2
Or 20                           2
Or 23                           1
Or 23; in some isoform(s)       1
Or 17                           1
Signal                          1
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
570,A9JLI2,SIGNAL,1..26,NaN,NaN,ECO:0000269,PubMed:30279544
598,A9JLI3,SIGNAL,1..31,NaN,NaN,ECO:0000269,PubMed:30279544
631,A9JLI5,SIGNAL,1..28,NaN,NaN,ECO:0000269,PubMed:30279544


In [184]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

ATP     853
NADP    627
FAD     624
NAD     588
GTP     452
FMN     238
AMP      68
PAPS     41
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
5631,Q7NXD4,NP_BIND,99..102,NaN,Substrate,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"
5632,Q7NXD4,NP_BIND,134..135,NaN,Substrate,"ECO:0000269,ECO:0007744","PubMed:24401123,PDB:2YB1"
5794,D7UER1,NP_BIND,201..209,NaN,FMN,"ECO:0000269,ECO:0007744","PubMed:26527149,PDB:4UWM"


In [185]:
feat_data = exp_data2[exp_data2.feature == "NP_BIND"]
print(feat_data['note'].value_counts(dropna=False).head(40))

ATP                                               853
NADP                                              627
FAD                                               624
NAD                                               588
GTP                                               452
FMN                                               238
AMP                                                68
PAPS                                               41
ADP                                                31
3',5'-cAMP                                         25
GMP                                                23
ATP-binding                                        22
UDP                                                21
CTP                                                18
ATP 1                                              17
ATP 2                                              17
dCTP binding                                       15
NADP 2                                             14
UTP                         

In [186]:
np_bind_mapping = {
    'ATP': {'ATP'},
    'NADP': {'NADP', 'NADP 2', 'NADP 2; for beta-ketoacyl reductase 2 activity', 'NADP 1', 'NADPH'},
    'NAD': {'NAD', 'NAD 1', 'NADH'},
    'FAD': {'FAD'},
    
}

In [187]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Periviscerokinin-1           65
Periviscerokinin-2           63
Pyrokinin-5                  51
Sulfakinin-1                 46
Hypertrehalosaemic factor    44
Periviscerokinin-3           42
Myomodulin-A                 19
Corazonin                    18
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
6482,P85078,PEPTIDE,1..21,PRO_0000278145,Tricyclic peptide MS-271,ECO:0000269,PubMed:8689231
9450,C0HJT4,PEPTIDE,1..63,PRO_0000436390,Toxin Tx7335,ECO:0000269,PubMed:27044983
11085,B3EWR3,PEPTIDE,1..>10,PRO_0000420493,Cytotoxin NN-32,ECO:0000269,PubMed:21530568


In [188]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Cleavage                            194
Not glycated                        109
Transition state stabilizer         106
Cleavage; by viral protease         102
Cleavage; by autolysis               92
Important for catalytic activity     85
Not glycosylated                     67
Interaction with DNA                 47
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1127,Q20655,SITE,240..241,NaN,Cleavage; by ced-3,ECO:0000269,PubMed:17371877
5391,Q2PS07,SITE,125,NaN,Critical for IgE-binding activity,ECO:0000269,PubMed:20407269
5435,P93198,SITE,113,NaN,Necessary for maximum IgE-binding by the immun...,ECO:0000269,PubMed:11799381


In [189]:
feat_data = exp_data2[exp_data2.feature == "SITE"]
print(feat_data['note'].value_counts(dropna=False).head(40))

Cleavage                                                  194
Not glycated                                              109
Transition state stabilizer                               106
Cleavage; by viral protease                               102
Cleavage; by autolysis                                     92
Important for catalytic activity                           85
Not glycosylated                                           67
Interaction with DNA                                       47
Cleavage; by 3CL-PRO                                       39
Cleavage; by protease 3C                                   35
Important for activity                                     33
(Microbial infection) Cleavage; by N.americanus apr- 2     33
Not ubiquitinated                                          31
Putative interfacial binding surface (IBS)                 29
Important for substrate specificity                        24
Cleavage; by caspase-3                                     22
Reactive

In [190]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN                                 2979
Removed in mature form               397
Activation peptide                   208
Leader sequence                       21
Removed by a carboxypeptidase         21
Removed by sortase                    14
Removed by a dipeptidylpeptidase      14
Cleaved by FlvT                       12
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
5165,P15457,PROPEP,22..37,PRO_0000032088,NaN,ECO:0000269,PubMed:16666238
5167,P15457,PROPEP,74..83,PRO_0000032090,NaN,ECO:0000269,PubMed:16666238
5171,P04403,PROPEP,23..36,PRO_0000032105,NaN,ECO:0000269,PubMed:3758080


In [191]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

1    121
2    120
3    103
4     96
5     80
6     73
7     64
8     57
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
4220,Q9UT08,REPEAT,12..50,NaN,HEAT 1,ECO:0000305,NaN
4221,Q9UT08,REPEAT,89..127,NaN,HEAT 2,ECO:0000305,NaN
4222,Q9UT08,REPEAT,206..244,NaN,HEAT 3,ECO:0000305,NaN


In [192]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Helical                                                2070
Beta stranded                                           183
Helical; Name=1                                         104
Helical; Name=2                                         101
Helical; Name=3                                          93
Helical; Name=4                                          92
Helical; Signal-anchor for type II membrane protein      87
Helical; Name=5                                          81
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
13184,P08195,TRANSMEM,185..205,NaN,Helical; Signal-anchor for type II membrane pr...,ECO:0000269,PubMed:30867591
16996,P41595,TRANSMEM,57..79,NaN,Helical; Name=1,"ECO:0000269,ECO:0000269,ECO:0000269","PubMed:23519215,PubMed:24357322,PubMed:28129538"
16998,P41595,TRANSMEM,91..113,NaN,Helical; Name=2,"ECO:0000269,ECO:0000269,ECO:0000269","PubMed:23519215,PubMed:24357322,PubMed:28129538"


In [193]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Nuclear localization signal    264
RxLR-dEER                      213
DDXXD motif                    146
Histidine box-2                104
Histidine box-1                104
Histidine box-3                102
DEAD box                        75
Effector region                 70
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
613,P18558,MOTIF,121..124,NaN,Prevents secretion from ER,ECO:0000269,PubMed:15016891
3925,Q8LF36,MOTIF,490..492,NaN,Microbody targeting signal,ECO:0000269,PubMed:19672620
6168,P55194,MOTIF,607..616,NaN,SH3-binding,ECO:0000269,PubMed:8438166


In [194]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Short neurotoxin 1           26
BDNF precursor form          19
Major capsid protein         11
Cytotoxin 1                  11
Collagen alpha-1(I) chain    10
Phosphoserine phosphatase    10
Cytotoxin 3                   8
Phospholipase A1              8
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
802,C0HLM8,CHAIN,1..>12,PRO_0000448941,Globulin 1 basic chain,ECO:0000305,NaN
1473,Q5ZMT0,CHAIN,1..255,PRO_0000223506,14-3-3 protein epsilon,ECO:0000305,NaN
3648,O18178,CHAIN,1..542,PRO_0000437754,Serine/threonine-protein phosphatase 2A regula...,ECO:0000305,NaN


In [195]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN    2533
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
806,C0HLR7,NON_CONS,13..14,NaN,NaN,ECO:0000303,Ref.1
807,C0HLR7,NON_CONS,19..20,NaN,NaN,ECO:0000303,Ref.1
808,C0HLR7,NON_CONS,40..41,NaN,NaN,ECO:0000303,Ref.1


In [196]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN    2088
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
804,C0HLM8,NON_TER,12,NaN,NaN,ECO:0000305,NaN
812,C0HLR7,NON_TER,1,NaN,NaN,ECO:0000303,Ref.1
813,C0HLR7,NON_TER,102,NaN,NaN,ECO:0000303,Ref.1


In [197]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

S-palmitoyl cysteine             464
S-diacylglycerol cysteine        312
N-myristoyl glycine              272
S-farnesyl cysteine               92
S-geranylgeranyl cysteine         80
N-myristoyl glycine; by host      42
S-palmitoyl cysteine; by host     38
GPI-anchor amidated serine        32
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
2221,P50927,LIPID,20,NaN,S-diacylglycerol cysteine,ECO:0000305,NaN
2225,P50928,LIPID,20,NaN,S-diacylglycerol cysteine,ECO:0000305,NaN
2229,Q84I68,LIPID,20,NaN,S-diacylglycerol cysteine,ECO:0000305,NaN


In [198]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Mitochondrion                    676
Chloroplast                      486
Thylakoid                         54
Chloroplast and mitochondrion     30
Peroxisome                         9
Amyloplast                         6
Apicoplast                         5
Hydrogenosome                      4
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
7631,Q9SZE1,TRANSIT,1..23,NaN,Mitochondrion,"ECO:0000255,ECO:0007744",",PubMed:22223895"
7695,P31937,TRANSIT,1..36,NaN,Mitochondrion,ECO:0000269,PubMed:8313870
30460,Q8W471,TRANSIT,1..66,NaN,Chloroplast,ECO:0000305,NaN


In [199]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

Carrier           66
Carrier 2         54
Carrier 1         54
EF-hand 1         36
UIM               30
Protein kinase    28
Carrier 3         28
EF-hand 2         21
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
18155,Q0CRQ5,DOMAIN,1617..1691,NaN,Carrier 1,"ECO:0000255,ECO:0000305","PROSITE-ProRule:PRU00258,PubMed:24412543"
18156,Q0CRQ5,DOMAIN,1736..1812,NaN,Carrier 2,"ECO:0000255,ECO:0000305","PROSITE-ProRule:PRU00258,PubMed:24412543"
38411,Q7PC84,DOMAIN,526..739,NaN,ABC transmembrane type-2 1,ECO:0000305,NaN


In [218]:
feat_data = man_data[man_data.feature == 'CARBOHYD']
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

N-linked (GlcNAc...) asparagine                            100234
N-linked (GlcNAc...) asparagine; by host                    13140
O-linked (GalNAc...) threonine                               2207
O-linked (GalNAc...) serine                                   535
O-linked (GlcNAc) serine                                      403
O-linked (GalNAc...) threonine; by host                       308
N-linked (GlcNAc...) (complex) asparagine                     283
N-linked (GlcNAc...) (high mannose) asparagine; by host       266
Name: note, dtype: int64


,taxon_id,uniprot_id,feature,sites,id,note,evidence,sources,manual_code,exp_code,ei_code,es_code
477,561445,P0C9J4,CARBOHYD,61,NaN,N-linked (GlcNAc...) asparagine; by host,ECO:0000255,NaN,True,False,False,False
484,10500,P0C9J5,CARBOHYD,61,NaN,N-linked (GlcNAc...) asparagine; by host,ECO:0000255,NaN,True,False,False,False
485,10500,P0C9J5,CARBOHYD,238,NaN,N-linked (GlcNAc...) asparagine; by host,ECO:0000255,NaN,True,False,False,False


In [200]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

C2H2-type 2              25
C4-type                  20
CCHC-type 2; atypical    16
CXXC-type                10
C2H2-type 1; atypical     9
C2H2-type 1               9
C2H2-type 2; atypical     7
C2H2-type 3; atypical     6
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
51948,Q06670,ZN_FING,148..197,NaN,C2HC BV-type,"ECO:0000255,ECO:0000269","PROSITE-ProRule:PRU01148,PubMed:23864639"
58878,P0A9Q5,ZN_FING,27..49,NaN,C4-type,ECO:0000305,NaN
231485,Q7Z2E3,ZN_FING,331..353,NaN,C2H2-type,"ECO:0000255,ECO:0000305,ECO:0000305","PROSITE-ProRule:PRU00042,PubMed:17276982,PubMe..."


In [201]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN                              55
Helical                          38
Pore-forming                     36
Discontinuously helical          23
Helical; Pore-forming             8
Note=Loop between two helices     3
Helical; Note=Signal for NS4B     3
Helical; Name=Pore helix 2        2
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
29293,Q15758,INTRAMEM,334..364,NaN,Discontinuously helical,ECO:0000305,PubMed:29872227
29297,Q15758,INTRAMEM,414..447,NaN,Discontinuously helical,ECO:0000305,PubMed:29872227
233324,Q96PS8,INTRAMEM,78..91,NaN,Discontinuously helical,ECO:0000269,PubMed:30420639


In [202]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

1                   46
2                   46
NaN                 20
3                   19
4                    7
2; high affinity     5
1; low affinity      5
High affinity        1
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
319211,Q12797,CA_BIND,90..102,NaN,NaN,ECO:0000305,PubMed:22586105
502691,O35505,CA_BIND,1534..1545,NaN,NaN,ECO:0000305,NaN
518901,P0DP23,CA_BIND,21..32,NaN,1,"ECO:0000269,ECO:0000269,ECO:0000269,ECO:000774...","PubMed:1474585,PubMed:25441029,PubMed:27564677..."


In [205]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

H-T-H motif              30
A.T hook                 26
NaN                      23
Homeobox 1                6
Homeobox 2                6
Homeobox 3                5
Non-target strand DNA     5
Homeobox 4                5
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
139814,Q9M2S3,DNA_BIND,88..100,NaN,A.T hook,ECO:0000305,NaN
139828,Q9LTA2,DNA_BIND,56..68,NaN,A.T hook,ECO:0000305,NaN
139834,Q9LZX7,DNA_BIND,59..71,NaN,A.T hook,ECO:0000305,NaN


In [206]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN    130
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
6345,O60239,COILED,44..90,NaN,NaN,ECO:0000269,PubMed:30217979
6346,O60239,COILED,97..145,NaN,NaN,ECO:0000269,PubMed:30217979
6347,O60239,COILED,154..200,NaN,NaN,ECO:0000269,PubMed:30217979


In [209]:
i += 1
feat_data = exp_data2[exp_data2.feature == cols[i]]
print(feat_data['note'].value_counts(dropna=False).head(8))
feat_data.head(3)

NaN    1
Name: note, dtype: int64


,uniprot_id,feature,sites,id,note,evidence,sources
1782872,Q8GXJ1,STRAND,150..153,NaN,NaN,ECO:0000305,NaN
